**TRAFFIC LIGHTS**

Le tutoriel utilise les fichiers :

* Experience pour des feux avec RL  :    
**utiliser** `examples/exp_configs/rl/singleagent/singleagent_traffic_light_grid.py`
* Experience pour des feux sans RL  :    
**utiliser** `examples/exp_configs/non_rl/traffic_light_grid.py`

Dans le fichier `traffic_light_grid.py`
* Network                           : class TrafficLightGridScenario
* Environment pour des feux avec RL : class TrafficLightGridEnv
* Environment pour des feux sans RL : class AccelEnv

On a 4 types de feux dans Flow:

1. [Statique](#Statique)
2. [Actionné](#Actionné)
3. [Actionné "Baseline"](#Actionné-"Baseline")
4. [RL](#RL)

In [1]:
from flow.core.params import NetParams
from flow.core.experiment import Experiment
from flow.networks.traffic_light_grid import TrafficLightGridNetwork
from flow.core.params import TrafficLightParams
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams, InFlows, SumoCarFollowingParams
from flow.core.params import VehicleParams
from flow.controllers import GridRouter
from flow.envs.ring.accel import AccelEnv, ADDITIONAL_ENV_PARAMS
import numpy as np

# TrafficLightGridNetwork

## grid_array
`grid_array` permet de créer la grille du network.
Ici, on crée une grille de 2 lignes et 3 colonnes.

In [2]:
inner_length = 300
long_length  = 500
short_length = 300
n = 2 # rows
m = 3 # columns
num_cars_left  = 21
num_cars_right = 21
num_cars_top   = 21
num_cars_bot   = 21
tot_cars = (num_cars_left + num_cars_right) * m + (num_cars_top + num_cars_bot) * n

grid_array = {"short_length": short_length, "inner_length": inner_length,
              "long_length": long_length, "row_num": n, "col_num": m,
              "cars_left": num_cars_left, "cars_right": num_cars_right,
              "cars_top": num_cars_top, "cars_bot": num_cars_bot}

## tl_logic
`tl_logic`dit à l'environnement si les feux sont controlés par RL, un pattern par défaut ou l'actuation de SUMO ("actuated"). 

In [3]:
tl_logic = TrafficLightParams()

## phases 

For more information about traffic light states, please refer to https://sumo.dlr.de/wiki/Simulation/Traffic_Lights#Signal_state_definitions


Pour ajouter les phases aux feux, on doit connaître leur node_id. Cette information est contenue dans la class Network de l'expérience (ainsi que dans le fichier `nod.xml` de l'expérience). 

Ici, on aura 6 noeuds: "center0" to "center5" :

     | | |
    -3-4-5-
     | | |
    -0-1-2-
     | | |

In [4]:
nodes  = ["center0", "center1", "center2", "center3", "center4", "center5"]
phases = [{"duration": "31", "state": "GrGr"},
          {"duration": "6", "state": "yryr"},
          {"duration": "31", "state": "rGrG"},
          {"duration": "6", "state": "ryry"}]

In [5]:
# ici, les 6 intersections ont le même ensemble de phases possibles (mais on peut mettre des phases différentes pour chaque noeud)
for node_id in nodes:
    tl_logic.add(node_id, tls_type="static", programID="1", offset=None, phases=phases)

## TrafficLightGridNetwork

In [6]:
additional_net_params = {"grid_array": grid_array, "speed_limit": 35,
                         "horizontal_lanes": 1, "vertical_lanes": 1,
                         "traffic_lights": True}

net_params = NetParams(additional_params=additional_net_params)

network = TrafficLightGridNetwork(name="grid", vehicles=VehicleParams(),
        net_params=net_params, initial_config=InitialConfig(), traffic_lights=tl_logic)

# Statique

Ceux sont des feux avec des phases pré-définis, ils ne peuvent pas s'ajuster dynamiquement au trafic et suivent un même schéma. On utilise pour ça le paramètre `baseline=False` à `TrafficLightParams`. 

In [7]:
tl_logic = TrafficLightParams(baseline=False)
phases = [{"duration": "31", "state": "GrGr"},
          {"duration": "6", "state": "yryr"},
          {"duration": "31", "state": "rGrG"},
          {"duration": "6", "state": "ryry"}]
tl_logic.add("center0", phases=phases, programID=1)

In [8]:
#!python simulate.py EXP_CONFIG

# Actionné

*SUMO prend en charge le contrôle du trafic actionné dit gap-based. Ce schéma de contrôle fonctionne en prolongeant les phases de trafic chaque fois qu'un flux continu de trafic est détecté. Il commute à la phase suivante après avoir détecté un intervalle de temps suffisant entre les véhicules successifs. Cela permet une meilleure répartition du temps vert entre les phases et affecte également la durée du cycle en réponse aux conditions de circulation dynamiques.*

* `minDur`: durée minimale de la phase
* `masDur`: durée maximale de la phase

_optionnels :_ 
* `maxGap`       : intervalle de temps maximal entre les véhicules successifs qui prolongera la phase actuelle
* `detectorGap`  : distance de temps entre le détecteur (généré automatiquement) et la ligne d'arrêt en secondes (à la vitesse maximale de chaque voie)
* `showDetectors`: active si oui ou non les détecteurs sont affichés dans sumo-gui
* `file`         : fichier dans lequel le détecteur inscrira les résultats
* `freq`         : période sur laquelle les valeurs collectées doivent être agrégées

In [9]:
tl_logic = TrafficLightParams(baseline=False)
phases = [{"duration": "31", "minDur": "8", "maxDur": "45", "state": "GrGr"},
          {"duration": "6", "minDur": "3", "maxDur": "6", "state": "yryr"},
          {"duration": "31", "minDur": "8", "maxDur": "45", "state": "rGrG"},
          {"duration": "6", "minDur": "3", "maxDur": "6", "state": "ryry"}]

tl_logic.add("center1", 
             tls_type="actuated", 
             programID="1", 
             phases=phases, 
             maxGap=5.0, 
             detectorGap=0.9, 
             showDetectors=False)

tl_logic.add("center2", tls_type="actuated")

In [10]:
#!python simulate.py EXP_CONFIG

# Actionné "Baseline"

Des feux actionnés "baseline" qui peuvent être utilisés pour n'importe quelle expérience sur une grille, ils ont été fine-tunés sur plusieurs itérations avec différents paramètres.

On peut les trouver dans la classe `TrafficLightParams` et la fonction `actuated_default()`.  

Pour les utiliser, il faut simplement initialiser TrafficLightParams avec `baseline` à `True` **(à tester ?)**
```python
tl_logic = TrafficLightParams(baseline=True)
additional_net_params = {"grid_array": grid_array, 
                         "speed_limit": 35,
                         "horizontal_lanes": 1, 
                         "vertical_lanes": 1,
                         "traffic_lights": True, 
                         "tl_logic": tl_logic}
```

In [11]:
#!python simulate.py traffic_light_grid True

In [12]:
tl_logic = TrafficLightParams(baseline=False)
phases = [{"duration": "31", "minDur": "8", "maxDur": "45", "state": "GrGrGrGrGrGr"},
          {"duration":  "6", "minDur": "3", "maxDur":  "6", "state": "yryryryryryr"},
          {"duration": "31", "minDur": "8", "maxDur": "45", "state": "rGrGrGrGrGrG"},
          {"duration":  "6", "minDur":  "3", "maxDur":  "6", "state": "ryryryryryry"}]
for node_id in nodes:
    tl_logic.add(node_id, phases=phases, programID=1, showDetectors=True, tls_type="actuated")
    
additional_net_params = {"grid_array": grid_array, "speed_limit": 35,
                         "horizontal_lanes": 1, "vertical_lanes": 1, 
                         "traffic_lights": True }

vehicles = VehicleParams()
vehicles.add(veh_id="human", routing_controller=(GridRouter, {}), car_following_params=SumoCarFollowingParams(min_gap=2.5, decel=7.5), num_vehicles=tot_cars)

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

initial_config = InitialConfig( spacing='custom', additional_params={'enter_speed': 10})

net_params = NetParams(additional_params=additional_net_params)

flow_params = dict( exp_tag   = 'grid-intersection',
                    env_name  = AccelEnv,
                    network   = TrafficLightGridNetwork,
                    simulator = 'traci',
                    sim       = SumoParams(sim_step=0.1, render=True ),
                    env       = EnvParams(horizon=1000,additional_params=ADDITIONAL_ENV_PARAMS.copy()),
                    net       = net_params,
                    veh       = vehicles,
                    initial   = initial_config,
                    tls       = tl_logic)

exp = Experiment(flow_params)

exp.run(1)

Round 0, return: 81.97996828345623
Average, std return:    81.97996828345623, 0.0
Average, std speed:     9.575547138951, 0.0
Total time:             94.97332215309143
steps/second:           12.50751178790073
vehicles.steps/second:  1977.3497380687202


{'returns': [81.97996828345623],
 'velocities': [array([ 0.19809524,  0.38616982,  0.56409597,  0.73187806,  0.8896388 ,
          1.03760368,  1.17608442,  1.30546245,  1.42617291,  1.53868986,
          1.64351295,  1.74115575,  1.83213587,  1.9169667 ,  1.99615076,
          2.07017451,  2.13950442,  2.20458411,  2.26583254,  2.32364284,
          2.37838195,  2.43039067,  2.47998415,  2.52745274,  2.57306295,
          2.61705873,  2.65966269,  2.70107746,  2.74148707,  2.78105827,
          2.81994184,  2.85827386,  2.89617692,  2.9337612 ,  2.97112557,
          3.00835858,  3.04553931,  3.08273828,  3.12001814,  3.15743444,
          3.19503618,  3.23286647,  3.27096298,  3.30935844,  3.34808103,
          3.38715478,  3.42659986,  3.4664329 ,  3.50666725,  3.54731316,
          3.58837801,  3.62986649,  3.6717807 ,  3.7141203 ,  3.75688261,
          3.80006267,  3.84365337,  3.88764543,  3.93202754,  3.97678633,
          4.02190643,  4.06737055,  4.11315948,  4.15925216,  4.2

# RL

Aucun élément `tl_logic` est nécessaire puisque le RL agent contrôle tous les paramètres.

Tous les noeuds sont reconnus comme des feux.

In [13]:
additional_net_params = {"speed_limit": 35, "grid_array": grid_array,
                         "horizontal_lanes": 1, "vertical_lanes": 1,
                         "traffic_lights": True}

On utilise l'environnement `TrafficLightGridEnv` qui permet d'appliquer des actions RL sur les feux via TraCI.

In [14]:
# !python train.py singleagent_traffic_light_grid.py

Setting `switch_time` enables more control over the RL experiment by preventing traffic lights from switching until `switch_time` timesteps have occurred. In practice, this can be used to prevent flickering.

In [15]:
additional_env_params = {"target_velocity": 50, "switch_time": 3.0}